In [ ]:
#            ORM-Python           DBMS          Database
#            SQLAlchemy          SQLite           FILE
#             engine -> connect
#                       dialect   *SQL(Params)     관리
#            MetaData(Object)
#            Table(Column)
#            sql(select, insert, ...)
#            MetaData(Table) <---> Database.Table
# 상위 클래스 상속 -> 클래스 선언 -> 객체 생성
# MetaData     -> 등록      -> 객체 생성(Row->insert), 객체 수정(Row->update)
# Class <----> Metadata(Table) <-----> Database.Table
# ---------------------------=> Session
# ORM은 데이터 관리가 목적. 

In [102]:
import sqlite3
import sqlalchemy

In [103]:
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.types import Integer, Text
from sqlalchemy.schema import ForeignKey, Column
from sqlalchemy.engine import create_engine

In [104]:
base = declarative_base()

In [105]:
base.metadata.tables

FacadeDict({})

In [106]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [107]:
dir(base)

['__abstract__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_sa_registry',
 'metadata',
 'registry']

In [108]:
# 서버랑 상관없이 클라이언트에서 작업, 싱크를 반드시 맞출 것
base.registry.dispose() # base에 등록된 class만 날린다
# base.metadata.clear()   # meta에 등록된 Table 객체를 날린다

In [109]:
class User(base):
    __tablename__ = 'USER'
    # __table_args__ = {'extend_existing': True}
    
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text, nullable=False)

In [110]:
base.metadata.tables

FacadeDict({'USER': Table('USER', MetaData(), Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<USER>, nullable=False), schema=None)})

In [111]:
user1 = User(name='사람1')

In [112]:
user1.pk, user1.name # Class->Instance 값 조회(DB와 무관함)

(None, '사람1')

In [113]:
base.metadata.create_all(engine)
# Class(base) 선언, Instance 생성 -> MetaData(Table 객체 등록) -> engine.dialect가 Table 객체를 SQL로 변환
# 

2025-03-07 10:26:49,111 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 10:26:49,111 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2025-03-07 10:26:49,112 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 10:26:49,113 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("USER")
2025-03-07 10:26:49,113 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 10:26:49,114 INFO sqlalchemy.engine.Engine 
CREATE TABLE "USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT NOT NULL, 
	PRIMARY KEY ("PK")
)


2025-03-07 10:26:49,114 INFO sqlalchemy.engine.Engine [no key 0.00027s] ()
2025-03-07 10:26:49,115 INFO sqlalchemy.engine.Engine COMMIT


In [114]:
# Instance <-------> 물리적DB
#        connect(engine)
sess = sessionmaker()
sess.bind = engine

In [115]:
s = sess()

In [116]:
s.bind = engine

In [117]:
s.add(user1)

In [118]:
s.dirty, s.is_modified(user1)
# session이 db에 기로 및 수정에 차이점이 있는지,
# session이 관리중인 객체가 변화가 있는지

(IdentitySet([]), True)

In [119]:
s.add(User(name='사람2'))

In [120]:
s.rollback()
s.commit()

In [121]:
user1.pk

In [122]:
s.close()
sess.close_all()

2025-03-07 10:27:43,078 INFO sqlalchemy.engine.Engine ROLLBACK


/var/folders/rp/63vh6wpj4w7_sq93lr8wwmm00000gn/T/ipykernel_4550/1955662118.py:2: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  sess.close_all()


In [123]:
session = sessionmaker()
sess = session()
sess.bind = engine

In [124]:
sess.add(user1)

In [125]:
sess.commit()

2025-03-07 10:27:48,448 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 10:27:48,450 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?)
2025-03-07 10:27:48,451 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ('사람1',)
2025-03-07 10:27:48,452 INFO sqlalchemy.engine.Engine COMMIT


In [126]:
user1.pk

2025-03-07 10:27:49,655 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 10:27:49,657 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2025-03-07 10:27:49,658 INFO sqlalchemy.engine.Engine [generated in 0.00079s] (1,)


1

In [127]:
result = sess.query(User).all()
# 세션이 관리하고 있는 객체들을 먼저 보고, 
# 메모리에서 충분히 관리할 수 있다고 판단이 되면 Execute -> fetchall

2025-03-07 10:27:51,248 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER"
2025-03-07 10:27:51,249 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ()


In [128]:
result[0] is user1

True

In [129]:
result[1].name

IndexError: list index out of range

In [69]:
sess.dirty

IdentitySet([])

In [74]:
sess.close()
base.registry.dispose()

2025-03-07 09:42:56,842 INFO sqlalchemy.engine.Engine ROLLBACK


In [76]:
base.metadata.tables

FacadeDict({'USER': Table('USER', MetaData(), Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<USER>, nullable=False), schema=None)})

In [77]:
base.metadata.clear()

In [78]:
sess.bind = engine

In [79]:
base.metadata.reflect(engine)

2025-03-07 09:44:06,645 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 09:44:06,646 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-07 09:44:06,646 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 09:44:06,647 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-07 09:44:06,648 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 09:44:06,648 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("USER")
2025-03-07 09:44:06,648 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 09:44:06,649 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2025-03-07 09:44:06,649 INFO sqlalchemy.engine.Engine [raw sql] ('USER',)
2025-03-07 09:44:06,650 INFO sqlalchemy.engine.Engine 

In [80]:
base.metadata.tables

FacadeDict({'USER': Table('USER', MetaData(), Column('PK', INTEGER(), table=<USER>, primary_key=True, nullable=False), Column('NAME', TEXT(), table=<USER>, nullable=False), schema=None)})

In [84]:
class NewUser(base):
    __table__ = base.metadata.tables['USER']
    
    NO = base.metadata.tables['USER'].c['PK']
    NAME = base.metadata.tables['USER'].c['NAME']
    # 클래스와 메타데이터.테이블 객체의 연결만 잘 시키면 됨.
    
    def print(self):
        print(self.NO, self.NAME)

/var/folders/rp/63vh6wpj4w7_sq93lr8wwmm00000gn/T/ipykernel_4550/4147149204.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.NewUser, and will be replaced in the string-lookup table.
  class NewUser(base):


In [85]:
r = sess.query(NewUser).all

In [86]:
r[0].print(), r[1].print()

TypeError: 'method' object is not subscriptable

In [87]:
sess.dirty

IdentitySet([])

In [91]:
sess.close()
base.registry.dispose()
base.metadata.clear()

In [93]:
engine = create_engine('sqlite:///coffee.db', echo=True)
sess.bind = engine
base.metadata.reflect(engine)

2025-03-07 09:52:01,316 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 09:52:01,316 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-07 09:52:01,317 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 09:52:01,318 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-07 09:52:01,318 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 09:52:01,318 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("ALBUM")
2025-03-07 09:52:01,318 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 09:52:01,319 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("ARTIST")
2025-03-07 09:52:01,319 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 09:52:01,319 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("CITY")
2025-03-07 09:52:01,320 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07

In [135]:
sess.close()
base.registry.dispose()
base.metadata.clear()
engine.connect().close()

2025-03-07 10:46:07,721 INFO sqlalchemy.engine.Engine ROLLBACK


In [136]:
engine = create_engine('sqlite:///coffee.db', echo=True)
sess.bind = engine

In [137]:
class City(base):
    __tablename__ = 'CITY'
    cno = Column('CNO', Integer, primary_key=True)
    name = Column('NAME', Text)
    
class supplier(base):
    __tablename__ = 'SUPPLIER'
    sno = Column('SNO', Integer, primary_key=True)
    name = Column('NAME', Text)
    cno = Column('CNO', Integer, ForeignKey('CITY.CNO'), nullable=False)
                                # 이 때는, meta.table에 등록되기 전이니까 물리적 DB의 테이블.컬럼
    
class Part(base):
    __tablename__ = 'PART'
    pno = Column('PNO', Integer, primary_key=True)
    name = Column('NAME', Text)
    
class Sells(base):
    __tablename__ = 'SELLS'
    no = Column('NO', Integer, primary_key=True)
    sno = Column('SNO', Integer, ForeignKey('SUPPLIER.SNO'), nullable=False)
    pno = Column('PNO', Integer, ForeignKey('PART.PNO'), nullable=False)
    price = Column('PRICE', Integer, default=0)

In [99]:
city = list()
for c in ['성북구', '강북구', '노원구']:
    city.append(City(name=c))
    
sess.add_all(city)

In [101]:
city

In [ ]:
city1 = sess.query(City).filter(City.name=='성북구').one()
city2 = sess.query(City).filter(City.name=='강북구').one()
city3 = sess.query(City).filter(City.name=='노원구').one()

2025-03-07 10:04:14,733 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 10:04:14,734 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("NAME") VALUES (?) RETURNING "CNO"
2025-03-07 10:04:14,735 INFO sqlalchemy.engine.Engine [generated in 0.00006s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('성북구',)
2025-03-07 10:04:14,736 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("NAME") VALUES (?) RETURNING "CNO"
2025-03-07 10:04:14,736 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('강북구',)
2025-03-07 10:04:14,737 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("NAME") VALUES (?) RETURNING "CNO"
2025-03-07 10:04:14,738 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('노원구',)
2025-03-07 10:04:14,739 INFO sqlalchemy.engine.Engine SELECT "CITY"."CNO" AS "CITY_CNO", "CITY"."NAME" AS "CITY_NAME" 
FROM "CITY" 
WHERE "CITY"."NAME" = ?
2025-03-07 10:04:14,740 INFO sqlalchemy.engine.Engine [generated in 0.0

MultipleResultsFound: Multiple rows were found when exactly one was required

In [ ]:
slist = [Supplier(name='지점1', cno=city.cno),
         Supplier(name='지점2', cno=city.cno)
         Supplier(name='지점1', cno=city.cno)
         Supplier(name='지점2', cno=city.cno)
         Supplier(name='지점3', cno=city.cno)]

sess.add_all(slist)
sess.commit()

세션에 등록했기 때문에 쿼리 없이 객체를 그냥 호출할 수 있게 됐다.

In [139]:
from sqlalchemy.orm import relationship

base.registry.dispose()

In [142]:
class City(base):
    __table__ = base.metadata.tables['CITY']
    slist = relationship('Supplier', back_populates='city', uselist=True)
    
class supplier(base):
    __table__ = base.metadata.tables['SUPPLIER']
    city = relationship('City', back_populates='slist')
    
# class Part(base):
#     __table__ = base.metadata.tables['PART']
    
# class Sells(base):
#     __table__ = base.metadata.tables['SELLS']

/var/folders/rp/63vh6wpj4w7_sq93lr8wwmm00000gn/T/ipykernel_4550/641223588.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.City, and will be replaced in the string-lookup table.
  class City(base):
/var/folders/rp/63vh6wpj4w7_sq93lr8wwmm00000gn/T/ipykernel_4550/641223588.py:5: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.supplier, and will be replaced in the string-lookup table.
  class supplier(base):


In [144]:
# c = sess.query(City).all()[0]